<a href="https://colab.research.google.com/github/petersolan/DL-CNN-Aerial-Images/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Feb  7 21:56:37 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 08162020
 09192020
'76 TOR international data experts to support UNCCD reporting .pdf'
 Adrian-Rosebrock-Deep-Learning-for.pdf
 ARiMR_2017_potencjal_Solan_Jarosław.xlsx
 ARiMR_2017_potencjal_Solan_Jarosław.xlsx.gsheet
'Arkusz kalkulacyjny bez tytułu.gsheet'
'BC Summary.pdf'
 Bluesky_Contract_of_Employment_Jaroslaw_Solan.pdf
 CJowo
'Colab Notebooks'
 CT
 DSC0001.JPG
 DSC0002.JPG
 DSC0003.JPG
 DSC0004.JPG
 DSC0005.JPG
 DSC0006.JPG
 DSC0007.JPG
 DSC0008.JPG
 DSC0009.JPG
 Emapsite
 FOTO_2014_lubelskie_czesc_III_i_IV_logistyka.gif
 FOTO_2014_lubelskie_czesc_III_i_IV_obiekty.gif
 FOTO_2015_monitoring.xlsx.gsheet
 Frampol_korekty.xlsx
 GeospatialInsight_EmployeeHandbookV2019.pdf
'GpsGate 2.6 + key.rar'
'GSI Employment Contract JSolan1119 (1).pdf'
'GSI Employment Contract JSolan1119.pdf'
 ID_Field

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
     

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 300/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Tree" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-02-07 21:58:03--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M   104MB/s    in 1.5s    

2022-02-07 21:58:04 (104 MB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/img1340.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
0 data/obj/img1040.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
1 data/obj/img1312.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
2 data/obj/img1226.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
3 data/obj/img3078.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
4 data/obj/img3110.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
5 data/obj/img3094.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
6 data/obj/img1392.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
7 data/obj/img3062.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
8 data/obj/img1200.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
9 data/obj/img1309.txt
0 0.5 0.5 0.5 0.5
0 0.5 0.5 0.5 0.5

['0', '0.5', '0.5', '0.5', '0.5']
10 data

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/img1200.jpg', 'data/obj/img1137.jpg', 'data/obj/img1054.jpg', 'data/obj/img1309.jpg', 'data/obj/img3122.jpg', 'data/obj/img1118.jpg', 'data/obj/img1020.jpg', 'data/obj/img1038.jpg', 'data/obj/img1047.jpg', 'data/obj/img3142.jpg', 'data/obj/img1170.jpg', 'data/obj/img2042.jpg', 'data/obj/img3146.jpg', 'data/obj/img1114.jpg', 'data/obj/img1306.jpg', 'data/obj/img3071.jpg', 'data/obj/img1108.jpg', 'data/obj/img1389.jpg', 'data/obj/img1063.jpg', 'data/obj/img1395.jpg', 'data/obj/img1381.jpg', 'data/obj/img3152.jpg', 'data/obj/img1124.jpg', 'data/obj/img1161.jpg', 'data/obj/img3090.jpg', 'data/obj/img1303.jpg', 'data/obj/img1052.jpg', 'data/obj/img1229.jpg', 'data/obj/img1241.jpg', 'data/obj/img1139.jpg', 'data/obj/img1033.jpg', 'data/obj/img1414.jpg', 'data/obj/img3062.jpg', 'data/obj/img1017.jpg', 'data/obj/img3081.jpg', 'data/obj/img3070.jpg', 'data/obj/img1074.jpg', 'data/obj/img1350.jpg', 'data/obj/img1280.jpg', 'data/obj/img1107.jpg', 'data/obj/img1289.jpg', 'data/obj/img31

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

In [ ]:
from keras.callbacks import *
filepath = "/content/gdrive/My Drive/yolov3/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.904451, iou_loss = 0.000000, total_loss = 0.904451 
 total_bbox = 14192, rewritten_bbox = 0.000000 % 
]2;226/300: loss=1.8 hours left=0.2
 226: 1.814419, 2.611466 avg loss, 0.000003 rate, 5.294170 seconds, 14464 images, 0.161820 hours left
Loaded: 0.000101 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.367949), count: 4, class_loss = 2.113877, iou_loss = 1.741735, total_loss = 3.855612 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.935989, iou_loss = 0.000000, total_loss = 0.935989 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.864285, iou_loss = 0.000000, total_loss = 0.864285 
 total_bbox = 14196, rewritten_bbox = 0.000000 % 
v3 (mse lo

In [ ]:
#!./darknet detector map custom/project.data custom/yolov3-project.cfg custom/yolov3-project_final.weights -dont_show -ext_output<custom/train.txt>result.txt